# PUBGコンペ
## PUBGって何？
まずはこちらを御覧ください
- <iframe width="560" height="315" src="https://www.youtube.com/embed/5dxequUSxjA?start=838" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

- <iframe width="560" height="315" src="https://www.youtube.com/embed/5dxequUSxjA?start=1353" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## どんなコンペか？
- １００人中何位になるか**０から１までの順位（０がビリ、１がトップ）**を予測する
- カーネルオンリールール、今書いている**カーネルのみ**でしか結果の提出は認められない。
- ０から１までの数値を予測するので**回帰問題**である
- 評価関数が平均絶対誤差（MAE）を使用、**小さければ小さいほどスコアが良い**

- 参考: [Otto解法説明カーネル](https://www.kaggle.com/wakamezake/neural-networks-product-classification-otto)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train_V2.csv")

In [ ]:
train.head()

In [ ]:
train.shape

# train_V2.csv
## カラムについて
- Id
プレーヤー個々に振られるID、インデックス
- groupId
チーム毎に振られるid、PUBGではソロ、デュオ、スクアッドなどのチームルールがある。
- matchId
プレイヤーはmatchid（試合毎に振られたid）毎に最大１００名の参加者が存在する。
- kills
キル数
- headshotKills
ヘッドショットキル数
- killPlace
１ゲーム内におけるキル数ランキング
- killPoints
kill数ベースのランキングに基づいたポイント数、プレイヤー毎に紐づく
０になっている人はまだランキング外ということ
<blockquote class="twitter-tweet" data-lang="ja"><p lang="en" dir="ltr">We already have an ELO based ranking system &amp; leaderboard on-game, and will be expanding it over the course of early access!</p>&mdash; PUBG (@PUBG) <a href="https://twitter.com/PUBG/status/845567597533835264?ref_src=twsrc%5Etfw">2017年3月25日</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

- winPoints killpointsと元にしたランキングが同じ、勝利数でプレイヤーに紐づく
- rankPoints killpointsと元にしたランキングが同じ、こちらはプレイヤーに紐づくランク
- killStreaks
短時間で連続でkillした数
- longestKill
killした際の敵との距離
- assists
自分が敵にダメージを与えた後自分以外が敵を倒した数
- damageDealt
トータルダメージ数、基本 kill × 100（プレイヤーHP）
- DBNOs
素手・格闘武器？で殴った敵の人数
- boosts ・heals
ブーストゲージ・体力をどれだけ回復したか
    - [【PUBG】回復アイテムとブーストの効果まとめ](https://vanillaice-fps.com/pubg/recovery-item-boost-item-effect)
    - [【PUBG】ブースト効果の解説と3種のブーストアイテムまとめ](https://chimolog.co/pubg-boost-guide/)
- rideDistance 車両に乗って走った距離
- walkDistance 走った距離
- matchDuration マッチにかかった時間 [秒]
- matchType 次セル参照
- maxPlace よくわからん
- numGroups 同マッチ内の
- revives チームメイトを復活させた数
- roadKills 車両で轢いて殺した敵の数
- swimDistance 泳いだ距離
- teamKills チームメイトをキルした数
- vehicleDestroys 車両を破壊した数
- weaponsAcquired 武器を拾った数


## 参考
[PUBG Finish Placement Prediction (Kernels Only) - data](https://www.kaggle.com/c/pubg-finish-placement-prediction/data)

In [ ]:
print(train["matchType"].unique())
print(len(train["matchType"].unique()))

In [ ]:
# どうやら欠損値が含まれている模様
train.isna().sum()

In [ ]:
train[train['winPlacePerc'].isna()]

In [ ]:
# 欠損値は削除！
train.drop(2744604, inplace=True)

In [ ]:
# チーターを見つける
# チーターの情報を学習に使ってしまうと正確な予測ができなくなる
# とりあえず動かずにkillしてるのはチーターとみなす
train['totalDistance'] = train['rideDistance'] + train['walkDistance'] + train['swimDistance']
train['killsWithoutMoving'] = ((train['kills'] > 0) & (train['totalDistance'] == 0))

In [ ]:
train[train['killsWithoutMoving'] == True].shape

In [ ]:
train[train['killsWithoutMoving'] == True]

In [ ]:
# 動かずにキルしているプレイヤーは削除
train.drop(train[train['killsWithoutMoving'] == True].index, inplace=True)

In [ ]:
# キル多すぎるやつがいるチーターだろ
print(train[train["kills"] > 30].shape)
train[train["kills"] > 30].head()

In [ ]:
# キル数多すぎるプレイヤーは削除
train.drop(train[train['kills'] > 30].index, inplace=True)

In [ ]:
# ヘッドショットキル / キル
# ヘッドショットレートを計算して、ヘッドショットレート1.0でキル数が多すぎるやつはチーター
train['headshot_rate'] = train['headshotKills'] / train['kills']
# 欠損値のために0で埋めておく
train['headshot_rate'] = train['headshot_rate'].fillna(0)

In [ ]:
# と思ったけど現実的にありえそうな範囲だった・・・
print(train[(train['headshot_rate'] == 1) & (train['kills'] > 9)].shape)
train[(train['headshot_rate'] == 1) & (train['kills'] > 9)]

In [ ]:
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor

In [ ]:
# catb_model = CatBoostRegressor() 
xgb_model = XGBRegressor()
# lgb_model = LGBMRegressor()

In [ ]:
# データ多すぎて学習が終わらないのでとりあえず 5万件
sample = train.sample(n=50000)

In [ ]:
x = sample.drop(columns=["Id", "groupId", "matchId", "matchType", "winPlacePerc"])
y = sample["winPlacePerc"]

In [ ]:
# catb_model.fit(x, y)
xgb_model.fit(x, y)
# lgb_model.fit(x, y)

In [ ]:
from xgboost import plot_importance

In [ ]:
# どの変数がモデルの精度向上に影響しているかを見てみる
plot_importance(xgb_model)